# Laboratório Elastic Search

# Atividade

## Contexto
O objetivo deste laboratório é explorar diferentes mecanismos de busca dentro e fora do Elastic Search!
Para isto, iremos explorar uma base de verificações de notícia de uma agência de checagem, chamada [Lupa](https://lupa.uol.com.br/).
O papel de uma agência de checagem é analisar uma notícia e verificar sua veracidade, gerar um veredito e retornar sua análise.

Neste laboratório, iremos nos preocupar apenas com o texto da análise e em como recuperá-los através de diferentes estratégias de busca.

## Tarefas
### Tarefa 1
- Cada uma das equipes receberá 2 queries fixas, chamadas de QF1 e QF2, e a tarefa do grupo será a criação de mais 2 queries, chamadas de QP1 e QP2. As queries devem seguir o formato de uma pergunta ou declaração textual e ela será usada na próxima tarefa para realizar buscas. As equipes podem se inspirar nas notícias fornecidas (presentes no csv da tarefa 2) para montá-las. Cada query deve ser composta por até 100 caracteres.
- A equipe deve enviar suas QP1 e QP2 em um formulário que será disponibilizado no classroom e no discord.

### Tarefa 2
- Após a criação das QP1 e QP2, cada equipe possuirá 4 queries (QF1; QF2; QP1 e QP2).
- Cada grupo irá realizar 4 tipos de busca. Uma busca léxica com BM25 (pelo ElasticSearch), uma busca semântica (pelo ElasticSearch), uma busca híbrida (manualmente utilizando as duas buscas anteriores) e uma estratégia de busca de sua preferência (neste lab ela será chamada de busca criativa), diferente das anteriores.
- As buscas léxica e semântica já estão implementadas, os grupos devem gerar a implementação das buscas híbridas e da busca criativa.
- A busca criativa será a busca usada para a competição!
- Cada grupo durante a implementação de suas buscas deve testar diferentes pré-processamentos nos dados para verificar como os resultados podem melhorar. Alguns pré-processamentos já foram implementados, mas os grupos não precisam se limitar aos pré-processamentos apresentados. Vocês podem buscar na internet implementações de outros pré-processamentos, criar padrões regex, etc. Observe que os dados precisarão ser reindexados no ElasticSearch sempre que os pré-processamentos forem modificados (inseridos, removidos ou mudados de ordem)!
- Enquanto vocês testam seus pré-processamentos e algoritmos de busca, ao analisar os resultados obtidos para uma determinada query, anotem o nível de relevância de cada documento lido em relação à query. Observe que esse procedimento pode ser realizado continuamente durante o processo de implementação, pois a relevância de um documento para uma query independe de implementação de buscadores. Essa anotação deve ser realizada (online) em uma planilha no google drive, que será fornecida pela organização do Lab. Caso mais de um integrante do grupo anote valores diferentes na planilha, não tem problemas. Nesse caso, iremos considerar a média dos valores.
    - A planilha tem o seguinte formato:
        - doc_id, query_id, relevance
        - 120, QP1, 2
        - 487, QP1, 1
        - ...
    - Cada busca deve possuir pelo menos 10 resultados anotados.
    - Não modifique o código que realiza o carregamento dos dados para que o doc_id seja consistente com os demais grupos!
    - A rotulação deve possui uma gradação em 3 níveis:
        - 0: Não é Relevante
        - 1: Pouco Relevante
        - 2: Muito Relevante 
- A entrega desta tarefa será feita através de um formulário para entregar um .zip contendo todo o repositório e o código utilizado autocontido (com listagem das dependências utilizadas no requirements.txt se necessário) e suas buscas implementadas. Este código será reexecutado, então organize o código para que ele possa ser reexecutado em outra máquina.
- Quando esgotar o prazo de envio da tarefa a planilha será bloqueada para modificações.
- A partir deste momento, nenhum grupo poderá alterar suas buscas novamente, então tenham ciência que esta será sua implementação final que será usada para a competição.

### Tarefa 3
- O grupo receberá (pelo discord) 3 queries adicionais (QA1; QA2 e QA3).
- A tarefa do grupo será a execução do MESMO CÓDIGO submetido na tarefa anterior com estas 3 queries e a anotação dos dados (a planilha de anotações será disponibilizada pelo discord).
- Nesta etapa, o grupo NÃO PODE FAZER ALTERAÇÕES nos pré-processamentos definidos ou nas implementações de suas buscas.
- Quando esgotar o prazo de envio da tarefa a planilha será bloqueada para modificações.
- IMPORTANTE: O seu código entregue na tarefa 2 será reexecutado com estas 3 queries (assim como você fez) para garantir que os resultados batem com os seus. Os grupos que mudarem suas buscas ou pré-processamentos na tarefa 3 terão sua nota penalizada e serão desclassificados da competição.

## Competição
A competição será realizada através da comparação dos resultados das buscas criativas em uma base de documentos (corpus) utilizada pela organização do Lab.
Para que você atinja um bom desempenho na competição, é importante se atentar ao seu processo de criação de queries e na qualidade da rotulação dos documentos, pois elas serão seus guias do quão boa sua busca está se saindo!

Como métrica de avaliação, esta competição irá utilizar a média do NDCG calculado para cada query. 

## Ferramental
Para executar este lab não é necessário uma máquina com GPU, mas a máquina deve ter capacidade de virtualização, além de possuir o Docker e o Python instalado.

Será utilizado o ElasticSearch (e opcionalmente o Kibana) no ambiente docker.

Para realizar o lab é recomendado o uso de Python 3.12, além de um venv ou ambiente conda.

Você deve instalar as dependências do requirements.txt.

## Detalhes
Ao executar o docker compose, além de ser levantado o ElasticSearch, também é levantada uma interface visual chamada Kibana.

O Kibana é um frontend para facilitar o acionamento de algumas operações do ElasticSearch e controle de configurações específicas, observar métricas etc. Ela pode ser utilizada para fins de debug. Para quem tiver curiosidade, acesse o URI: http://localhost:5601 após levantar o serviço com o docker compose.

## Dúvidas
Caso tenham dúvidas, é só entrar em contato pelo nosso servidor do Discord.

-----------------------
# Código


## Passos de preparação do ambiente

### Imports

In [1]:
import zipfile
import requests
import os
import pandas as pd
from datetime import datetime
import re
import warnings
import subprocess
warnings.filterwarnings("ignore")
from collections import OrderedDict

from elasticsearch import Elasticsearch

from sentence_transformers import SentenceTransformer

import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
import sklearn

nltk.download('punkt_tab')
nltk.download('stopwords')

import spacy
import unidecode

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/helenavm/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/helenavm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Passo 0: Subir Stack do Elastic (ElasticSearch e Kibana)

In [2]:
!python -m spacy download pt_core_news_sm # Caso o comando não funcione, execute-o no terminal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 10.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [3]:
subprocess.call(["docker", "compose", "up", "-d"])
# Se esse comando falhar ou retornar 1, execute-o diretamente no terminal para identificar o erro.
# PS: O docker deve estar instalado e rodando

time="2025-03-25T18:45:52-03:00" level=warning msg="/home/helenavm/source/LabElasticSearch---Group8/docker-compose.yaml: `version` is obsolete"
 Container labelasticsearch---group8-setup-1  Created
 Container labelasticsearch---group8-es01-1  Running
 Container labelasticsearch---group8-kibana-1  Running
 Container labelasticsearch---group8-setup-1  Starting
 Container labelasticsearch---group8-setup-1  Started
 Container labelasticsearch---group8-setup-1  Waiting
 Container labelasticsearch---group8-setup-1  Healthy
 Container labelasticsearch---group8-es01-1  Waiting
 Container labelasticsearch---group8-es01-1  Healthy


0

### Passo 1: Baixar dados do Lupa

In [4]:
# Base de dados de notícias da Lupa
url = "https://docs.google.com/uc?export=download&confirm=t&id=1W067Md2EbvVzW1ufzFg17Hf7Y9cCZxxr"
filename = "articles_lupa_lab_elasticsearch.zip"
data_path = "data"
zip_file_path = f"{data_path}/{filename}"

os.makedirs(data_path, exist_ok=True)

# Baixa o zip
with open(zip_file_path, "wb") as f:
    f.write(requests.get(url, allow_redirects=True).content)

# Extrai o csv do zip
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(data_path)
    
output_file = f"{data_path}/articles_lupa.csv"
assert os.path.exists(output_file)

### Passo 2: Pré-processar os dados e gerar embeddings

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer



In [48]:
# Implementações de pré-processamentos de texto. Modifiquem, adicionem, removam conforme necessário.
class Preprocessors:
    STOPWORDS = set(nltk.corpus.stopwords.words('portuguese'))
    
    def __init__(self, customSW=None):
        self.stemmer = PorterStemmer()
        self.spacy_nlp = spacy.load("pt_core_news_sm") # Utiliza para lematização
        self.custom_stop_words = customSW
        
    # Remove stopwords do português
    def remove_stopwords(self, text):
        # Tokeniza as palavras
        tokens = word_tokenize(text)

        sw = self.STOPWORDS
        if not (self.custom_stop_words is None):
            sw = self.custom_stop_words
        # Remove as stop words
        tokens = [word for word in tokens if word not in sw]

        return ' '.join(tokens)
    
    # Realiza a lematização
    def lemma(self, text):
        return " ".join([token.lemma_ for token in self.spacy_nlp(text)])
    
    # Realiza a stemização
    def porter_stemmer(self, text):
        # Tokeniza as palavras
        tokens = word_tokenize(text)

        for index in range(len(tokens)):
            # Realiza a stemização
            stem_word = self.stemmer.stem(tokens[index])
            tokens[index] = stem_word

        return ' '.join(tokens)

    # Transforma o texto em lower case
    def lower_case(self, str):
        return str.lower()

    # Remove urls com regex
    def remove_urls(self, text):
        url_pattern = r'https?://\S+|www\.\S+'
        without_urls = re.sub(pattern=url_pattern, repl=' ', string=text)
        return without_urls

    # Remove números com regex
    def remove_numbers(self, text):
        number_pattern = r'\b(?!\d{4}\b|\b[12]?\d{1,2}\b|\b[1-2]?\d{3}\b)\d+\b'
        without_number = re.sub(pattern=number_pattern,
    repl=" ", string=text)
        return without_number

    # Converte caracteres acentuados para sua versão ASCII
    def accented_to_ascii(self, text):
        text = unidecode.unidecode(text)
        return text

    # Remove símbolos com regex
    def remove_symbols(self, text):
        symbols_pattern = r'[^\w\s]'
        without_symbols = re.sub(symbols_pattern, repl='', string=text)
        return without_symbols

In [46]:
def get_custom_stop_words(words, limiar_percent=0.1):
    preprocessor = Preprocessors()
    stop_words_preprocessing_steps = [
        preprocessor.lower_case,
        preprocessor.remove_urls,
        preprocessor.accented_to_ascii,
        preprocessor.remove_numbers,
        preprocessor.remove_symbols
    ]

    for preprocess in stop_words_preprocessing_steps:
        words = preprocess(words)
    
    tf = {}
    for word in words.split(' '):
        if tf.get(word):
            tf[word] += 1
        else:
            tf[word] = 1

    words = sorted(tf.items(), key=lambda item: item[1], reverse=True)

    limiar = len(words) * limiar_percent / 100

    return [x for x, _ in words[:int(limiar)]]

In [ ]:
# Carregar o modelo gerador de embeddings
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Caminho para salvar o dataframe de notícias
data_df_path = "data/data_df.pkl"

# Selecione diferentes pré-processamentos
# Exemplo:
"""
preprocessor = Preprocessors()
preprocessing_steps = [
    preprocessor.remove_urls,
    preprocessor.remove_stopwords,
]
"""

RECREATE_DF = True

# Cria o data frame se ele já existir ou se a variável RECREATE_INDEX for verdadeira
# Ou (exclusivo) carrega o dataframe salvo
if not os.path.exists(data_df_path) or RECREATE_DF:    
    df = pd.read_csv(output_file, sep=";")[["Título", "Texto", "Data de Publicação"]]

    all_words = ' '.join(df["Texto"].dropna().astype(str).str.split().explode())
    custom_stop_words = get_custom_stop_words(all_words)

    preprocessor = Preprocessors(custom_stop_words)
    preprocessing_steps = [
        preprocessor.lower_case,
        preprocessor.remove_urls,
        preprocessor.accented_to_ascii,
        preprocessor.remove_numbers,
        preprocessor.remove_stopwords,
        preprocessor.lemma
    ]


    df["Data de Publicação"] = df["Data de Publicação"].apply(lambda str_date: datetime.strptime(str_date.split(" - ")[0], "%d.%m.%Y"))
    df.sort_values("Data de Publicação", inplace=True, ascending=False)
    df["Embeddings"] = [None] * len(df)
    df["doc_id"] = df.reset_index(drop=True).index

    for i, row in df.iterrows():
        texto_completo = row["Texto"].strip() + "\n" + row["Título"].strip()
        
        df.at[i, "Texto completo"] = texto_completo
        texto_processado = texto_completo
        for preprocessing_step in preprocessing_steps:
            texto_processado = preprocessing_step(texto_processado)
        
        df.at[i, "Texto processado"] = texto_processado
        embeddings = model.encode(texto_completo).tolist()
        df.at[i, "Embeddings"] = embeddings
        
    print("Geração de embeddings finalizada.")
    
    with open(data_df_path, "wb") as f:
        df.to_pickle(f) 
else:
    with open(data_df_path, "rb") as f:
        df = pd.read_pickle(f)
    print("Dataframe carregado de arquivo.")

74


### Passo 3: Indexar dados no ElasticSearch (Lembrem-se de reindexar os dados se os pré-processamentos mudarem)

In [68]:
es = Elasticsearch(
    hosts = [{'host': "localhost", 'port': 9200, "scheme": "https"}],
    basic_auth=("elastic","elastic"),
    verify_certs = False,
)

In [69]:
RECREATE_INDEX = True

index_name = "verificacoes_lupa"

# Se a flag for True e se o índice existir, ele é deletado
if RECREATE_INDEX and es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"Índice '{index_name}' deletado.")

# Cria o índice e popula com os dados
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, mappings={
        "properties": {
            "doc_id": {"type": "integer"},
            "full_text": {"type": "text"},
            "processed_text": {"type": "text"},
            "embeddings": {"type": "dense_vector", "dims": 384}
        }
    })
    print(f"Índice '{index_name}' criado.")
    
    for i, row in df.iterrows():
        es.index(index=index_name, id=row["doc_id"], body={
            "doc_id": row["doc_id"],
            "full_text": row["Texto completo"],
            "processed_text": row["Texto processado"],
            "embeddings": row["Embeddings"]
        })
    print("Índice preenchido.")

print("Indexação finalizada.")

Índice 'verificacoes_lupa' deletado.
Índice 'verificacoes_lupa' criado.
Índice preenchido.
Indexação finalizada.


## Tarefas

### Tarefa 1: Criar query
Crie as queries da tarefa 1 (QP1 e QP2), as queries devem ser perguntas ou declarações de até 100 caracteres.

Inspire-se nas notícias presente no csv (data/articles_lupa.csv) para gerar queries interessantes.

Submetam as queries ao formulário.

### Tarefa 2: Implementação e realização das buscas
Agora que você montou suas queries, realize cada uma das buscas para cada uma das queries (QF1; QF2; QP1 e QP2).

In [70]:
# Estas serão as queries QF1 e QF2
with open("data/queries_fixadas.txt", "r") as f:
    queries_fixadas = [line.strip() for line in f.readlines()]
    assert len(queries_fixadas) == 2
    QF1 = queries_fixadas[0]
    QF2 = queries_fixadas[1]
    
# Preencha aqui as queries do grupo
QP1 = "Como se proteger de novos golpes online"
QP2 = "Existem leis anti discriminação e violência política no Brasil?"

queries = OrderedDict()
queries["QF1"] = QF1
queries["QF2"] = QF2
queries["QP1"] = QP1
queries["QP2"] = QP2

for q in ["Haverá cobrança na transferência bancária digital a partir de agora","As vacinas da covid19 causaram mortes","Como se proteger de novos golpes online", "Existem leis anti discriminação e violência política no Brasil?"]:
    print(q)   
    for preprocessing_step in preprocessing_steps:
        q = preprocessing_step(q)
        print(q)

Haverá cobrança na transferência bancária digital a partir de agora
haverá cobrança na transferência bancária digital a partir de agora
haverá cobrança na transferência bancária digital a partir de agora
havera cobranca na transferencia bancaria digital a partir de agora
havera cobranca na transferencia bancaria digital a partir de agora
havera cobranca transferencia bancaria digital partir agora
haverar cobranca transferencia bancario digital partir agora
As vacinas da covid19 causaram mortes
as vacinas da covid19 causaram mortes
as vacinas da covid19 causaram mortes
as vacinas da covid19 causaram mortes
as vacinas da covid19 causaram mortes
vacinas covid19 causaram mortes
vacina covid19 causar morte
Como se proteger de novos golpes online
como se proteger de novos golpes online
como se proteger de novos golpes online
como se proteger de novos golpes online
como se proteger de novos golpes online
proteger novos golpes online
proteger novo golpe onliner
Existem leis anti discriminação 

#### Busca Léxica

In [71]:
# Implementação de busca esparsa (léxica) com BM25
def lexical_search(queries: dict[str, str]):
    lexical_results = {}
    for query_id, query in queries.items():
        
        # Pré-processa os dados
        for preprocessing_step in preprocessing_steps:
            query = preprocessing_step(query)
        
        search_query = {
            "size": 10,
            "query": {
                "match": {
                    "processed_text": query
                }
            }
        }

        # Realiza a busca
        response = es.search(index=index_name, body=search_query)
        
        hits_results = []
        # Recupera os resultados
        for hit in response["hits"]["hits"]:
            hits_results.append((hit["_source"]["doc_id"], hit["_score"]))
        lexical_results[query_id] = hits_results
        
    return lexical_results

#### Busca Semântica

In [78]:
# Realiza busca semântica (densa) com KNN exato
def semantic_search(queries: dict[str, str]):
    semantic_results = {}
    
    for query_id, query in queries.items():
        # Aplica todos os pré-processamentos aos dados
        for preprocessing_step in preprocessing_steps:
            query = preprocessing_step(query)
            
        query_vector = model.encode(query).tolist()
        
        
        search_query = {
            "size": 10,
            "query": {
                "script_score": {
                    "query": {"match_all": {}},
                    "script": {
                        "source": "cosineSimilarity(params.query_vector, 'embeddings') + 1.0",
                        "params": {"query_vector": query_vector}
                    }
                }
            }
        }

        # Realiza a busca
        response = es.search(index=index_name, body=search_query)
        
        hits_results = []
        # Recupera top 10 resultados
        for hit in response["hits"]["hits"]:
            hits_results.append((hit["_source"]["doc_id"], hit["_score"]))
            
        semantic_results[query_id] = hits_results

    return semantic_results

#### Busca Híbrida

In [79]:
# Busca híbrida ou RRF. Implemente sua solução aqui. Você pode realizar as duas buscas anteriores (léxica e semântica) como base para formar a busca híbrida.
def hybrid_search(queries: dict[str, str]):
    lexical_results = lexical_search(queries)
    restricted_semantic_results = {}

    for query_id, query in queries.items():
        # Aplica pré-processamentos
        for preprocessing_step in preprocessing_steps:
            query = preprocessing_step(query)
        
        # Codifica query em vetor denso
        query_vector = model.encode(query).tolist()

        # Obtém os doc_ids da busca léxica
        lexical_doc_ids = [doc_id for doc_id, _ in lexical_results.get(query_id, [])]
        
        if not lexical_doc_ids:
            restricted_semantic_results[query_id] = []
            continue

        # Monta query semântica restrita aos doc_ids
        search_query = {
            "size": 10,
            "query": {
                "script_score": {
                    "query": {
                        "terms": {
                            "doc_id": lexical_doc_ids
                        }
                    },
                    "script": {
                        "source": "cosineSimilarity(params.query_vector, 'embeddings') + 1.0",
                        "params": {"query_vector": query_vector}
                    }
                }
            }
        }

        response = es.search(index=index_name, body=search_query)
        
        hits_results = []
        for hit in response["hits"]["hits"]:
            hits_results.append((hit["_source"]["doc_id"], hit["_score"]))
        
        restricted_semantic_results[query_id] = hits_results

    return restricted_semantic_results

#### Busca Criativa

In [93]:
# Implemente sua própria estratégia de busca, podendo ela ser esparsa, densa ou híbrida. Implemente algo como "more_like_this", "BM35", "fuzzy" etc.
def creative_search(queries: dict[str, str]):
    ## TODO: Implementar busca híbrida
    pass

#### Execução das buscas

In [80]:
search_functions = [
    ("lexical", lexical_search),
    ("semantic", semantic_search),
    ("hybrid", hybrid_search),
    #("creative", creative_search)
]

def run_all_searches(queries: dict[str, str]):
    all_search_results = {}
    for search_name, search_function in search_functions:
        results = search_function(queries)
        all_search_results[search_name] = results
    return all_search_results

#### Analise os resultados da busca e aprimore a busca!

In [81]:
all_search_results = run_all_searches(queries)
search_results_df = pd.DataFrame(all_search_results)
search_results_df

,lexical,semantic,hybrid
QF1,"[(1920, 20.649078), (447, 18.345564), (623, 15.7814865), (1660, 15.456156), (3479, 14.8354225), (1991, 14.763208), (1766, 14.344), (88, 14.1229), (3825, 13.533148), (1491, 13.369331)]","[(1491, 1.6058515), (1920, 1.5239503), (82, 1.5190855), (1660, 1.5080773), (1606, 1.500781), (1991, 1.5001796), (1087, 1.4997236), (2255, 1.4974422), (307, 1.4880704), (2130, 1.4849763)]","[(1491, 1.6058515), (1920, 1.5239503), (1660, 1.5080773), (1991, 1.5001796), (3825, 1.4232183), (1766, 1.4219997), (447, 1.3710871), (88, 1.3564069), (623, 1.328193), (3479, 1.2386197)]"
QF2,"[(4618, 12.920881), (4664, 11.085611), (4032, 11.06958), (2972, 10.93523), (1228, 10.587513), (3761, 10.5724745), (1109, 10.487575), (3249, 10.461214), (3710, 10.44688), (1183, 10.409498)]","[(3710, 1.8208842), (3461, 1.8018768), (2972, 1.7968816), (3409, 1.793333), (3692, 1.7650025), (3249, 1.7608975), (2995, 1.7585142), (3092, 1.7525693), (3208, 1.7492588), (2861, 1.7490219)]","[(3710, 1.8208842), (2972, 1.7968816), (3249, 1.7608975), (3761, 1.7402564), (1228, 1.673691), (1183, 1.6664984), (4664, 1.5581268), (4032, 1.5524788), (1109, 1.5515052), (4618, 1.3420492)]"
QP1,"[(844, 13.145399), (4168, 10.680053), (122, 10.611412), (306, 10.090786), (138, 9.955878), (150, 9.319978), (1450, 9.102754), (3818, 9.089081), (2670, 9.072411), (1553, 8.929828)]","[(679, 1.6366354), (2700, 1.6223811), (1898, 1.6191282), (1118, 1.6147891), (1547, 1.6095119), (1688, 1.6089125), (1070, 1.6080425), (2548, 1.6066722), (1817, 1.6003315), (462, 1.6002744)]","[(1553, 1.5401314), (138, 1.5322112), (306, 1.499848), (1450, 1.4586122), (150, 1.4332399), (3818, 1.403076), (4168, 1.3438623), (844, 1.3353448), (2670, 1.3183848), (122, 1.3100771)]"
QP2,"[(1219, 15.603304), (295, 15.412268), (2856, 15.29161), (1357, 15.187067), (2632, 14.911072), (1016, 14.576488), (1400, 14.485383), (821, 13.825228), (1584, 13.726249), (2903, 13.550341)]","[(2970, 1.680339), (1604, 1.6665958), (3850, 1.6664263), (2287, 1.6659178), (1162, 1.665857), (1016, 1.6655575), (2023, 1.6639389), (2158, 1.6614459), (2004, 1.6585871), (4007, 1.6569201)]","[(1016, 1.6655575), (295, 1.6498008), (1219, 1.6413367), (821, 1.5782073), (2903, 1.4499571), (2632, 1.400956), (1400, 1.395207), (2856, 1.3552467), (1584, 1.2498955), (1357, 1.195461)]"


In [82]:
def generate_exploded_df(search_results_df):
    exploded_search_results_df = pd.concat([search_results_df[col].explode() for col in search_results_df.columns], axis=1)
    exploded_search_results_df = exploded_search_results_df.apply(lambda l: [doc_id for doc_id, _ in l])
    return exploded_search_results_df

def generate_found_docs_text_df(exploded_search_results_df, all_docs_df):
    # Recupera os ids únicos dos documentos
    documents_ids = set(exploded_search_results_df.to_numpy().flatten().tolist())

    # Salva os textos e os ids dos documetnos que foram encontrados ems usas buscas
    documents_df = all_docs_df[all_docs_df["doc_id"].isin(documents_ids)][["Texto processado", "doc_id", "Texto completo"]]
    return documents_df

exploded_df = generate_exploded_df(search_results_df)
found_docs_text_df = generate_found_docs_text_df(exploded_df, all_docs_df=df)

def save_results_to_file(exploded_df: pd.DataFrame,
                         found_docs_text_df: pd.DataFrame,
                         exploded_df_save_filepath: str = "data/search_results.csv",
                         found_docs_text_save_filepath: str = "data/documents.csv"):
    exploded_df.to_csv(exploded_df_save_filepath)
    found_docs_text_df.to_csv(found_docs_text_save_filepath)
    print("Resultados das buscas salvos em 'data/search_results.csv'.")
    print("Documentos de interesse salvos em 'data/documents.csv'.")
    
save_results_to_file(exploded_df, found_docs_text_df)

Resultados das buscas salvos em 'data/search_results.csv'.
Documentos de interesse salvos em 'data/documents.csv'.


In [84]:
pd.set_option('expand_frame_repr', True)
pd.set_option('display.max_colwidth', 1000)
# print(exploded_df.head())
print(df.columns)

os.makedirs('output3', exist_ok=True)
for query in queries:
    os.makedirs(f'output3/{query}', exist_ok=True)
    
    # Get all rows for this query
    query_rows = exploded_df.loc[query]
    
    # Process each ranking method (column)
    for col in ['lexical', 'semantic', 'hybrid']:  # or use exploded_df.columns
        rel_df = {'DocID': [], 'Título': [], 'Texto completo': []}
        
        # Get all doc IDs for this query+column combination
        doc_ids = query_rows[col].tolist()
        
        for doc_id in doc_ids:
            # Find matching document
            doc = df[df['doc_id'] == doc_id]
            if not doc.empty:
                rel_df['DocID'].append(doc_id)
                rel_df['Título'].append(doc['Título'].values[0])
                rel_df['Texto completo'].append(doc['Texto completo'].values[0])
        
        # Save to CSV
        pd.DataFrame(rel_df).to_csv(f'output3/{query}/{col}.csv', index=False)

Index(['Título', 'Texto', 'Data de Publicação', 'Embeddings', 'doc_id',
       'Texto completo', 'Texto processado'],
      dtype='object')


#### Anote as relevâncias na sua planilha!

### Tarefa 3: Reexecutar a busca para as novas queries e rotular os dados

In [ ]:
# Preencha aqui com as queries adicionais do seu grupo
QA1 = XXXXXXXXX
QA2 = XXXXXXXXX
QA3 = XXXXXXXXX

queries = OrderedDict()
queries["QF1"] = QF1
queries["QF2"] = QF2
queries["QA1"] = QA1
queries["QA2"] = QA2
queries["QA3"] = QA3


all_search_results = run_all_searches(queries)
search_results_df = pd.DataFrame(all_search_results)
search_results_df

In [ ]:
exploded_df = generate_exploded_df(search_results_df)
found_docs_text_df = generate_found_docs_text_df(exploded_df, all_docs_df=df)

def save_results_to_file(exploded_df: pd.DataFrame,
                         found_docs_text_df: pd.DataFrame,
                         exploded_df_save_filepath: str = "data/search_results.csv",
                         found_docs_text_save_filepath: str = "data/documents.csv"):
    exploded_df.to_csv(exploded_df_save_filepath)
    found_docs_text_df.to_csv(found_docs_text_save_filepath)
    print("Resultados das buscas salvos em 'data/search_results.csv'.")
    print("Documentos de interesse salvos em 'data/documents.csv'.")
    
save_results_to_file(exploded_df, found_docs_text_df)
exploded_df

#### Anote os resultados na sua planilha!

#### A competição utilizará o NDCG médio por query para computar seu desempenho.